In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import gensim
print("gensim_version",(gensim.__version__))  
from gensim.models import Word2Vec
import pickle
import tqdm
from tqdm import tqdm

gensim_version 4.1.2


In [2]:
df_all = pd.read_csv("df_sw_tok_low_punc_lemm_v7.csv").rename(columns = {'doi_x':'doi'})

In [3]:
def add_target_opencitatins_marginal(target_year,df,target_col_name,get_cum=False): 
    """
    Create target flag for dataset of doi (df) for selected target year.
    Is automatically derived from all doi of each year of publication in df -> 
    -> median of citations of articles published in year of publication of the articles. 
    Target = higly and lowly cited articles based on median of the year of publication. Derived from 
             marginal citations of the target_year which express relevance of articles that year.
    Needed columns: doi, Year (of publication)
    
    """    
    df_cit_year = pd.read_csv("citationcounts_oci_revised_year.csv",error_bad_lines=False,encoding="utf-8")
    df_cit_year = df_cit_year.rename(columns={'count_opencitations': 'Year_of_citations',"year;;;;;;":"OpenCitations","doi_x":"doi"})
    df_cit_year['OpenCitations'] =  df_cit_year['OpenCitations'].str.extract(r'(\d+)', expand=False)
    df_cit_year = df_cit_year[df_cit_year['Year_of_citations']!="?"]
    df_cit_year = df_cit_year[df_cit_year.pmc.notnull()]
    df_cit_year = df_cit_year[['doi',"Year_of_citations",'OpenCitations']].dropna()
    df_cit_year['OpenCitations'] = df_cit_year['OpenCitations'].astype(int)
    df_cit_year_pivot = pd.pivot_table(df_cit_year, values='OpenCitations', index=['doi'], columns=['Year_of_citations'], aggfunc=np.sum)

     # filter citations for target year
    df_cited_in_target_year = df_cit_year_pivot[[num for num in list(df_cit_year_pivot.columns) if int(num)==target_year]] 
    
    # for target - needed to have not null citations in target year (citations starting with 1 citations)
    df_cited_in_target_year = df_cited_in_target_year[pd.notnull(df_cited_in_target_year[str(target_year)])] 
    df_cited_in_target_year.columns = ["OpenCitations"] # rename column of citations to target column
   
    if get_cum == True:
        # filter citations for target years
        df_cited_in_target_year = df_cit_year_pivot
        df_cit_year_pivot["OpenCitations"] = df_cited_in_target_year.sum(axis=1)
 
    
    # derive target based on median of year of publication 
    df_joined = df.merge(df_cited_in_target_year,on="doi",how="inner") # joined with data to have there additional information

    median_of_year = df_joined[["Year","OpenCitations"]].groupby("Year"
                      ).median().reset_index().rename(columns={"OpenCitations": 'median_cit'}) 

    df_with_med =  df_joined.merge(median_of_year, on="Year", how="left")
    df_with_med[target_col_name] =  np.where( df_with_med["OpenCitations"]> df_with_med["median_cit"],1, 0)
    print(df_with_med[target_col_name].value_counts())
    return df_with_med

In [4]:
def tokenized_column(string_col
                    ):  
    tokenized_list=[]
    for i in list(string_col):
        i = str(i)
        li = list(i.split(' '))
        tokenized_list.append(li)
    return tokenized_list

In [5]:
def transform_to_document_vector(text_col_tokenized,
                                 model,
                                 index_col_list,
                                 agg_func = "avg" # or sum
                                ):
    doc_vectors_final = []
    for doc in tqdm(text_col_tokenized):
        doc_vectors = []
        word_names = []
   
    
        for word in doc:
    #        if word in model.wv.vocab:
             if word in model.wv.index_to_key:
    #           doc_vectors.append(model[word])
                doc_vectors.append(model.wv[word])
                word_names.append(word)
                
        doc_vectors_pd = pd.DataFrame(doc_vectors)
        doc_vectors_pd.index = word_names
        
        if agg_func == "avg":
            doc_vector = list(doc_vectors_pd.mean())
        
        if agg_func == "sum":
            doc_vector = list(doc_vectors_pd.sum())
            
        doc_vectors_final.append(doc_vector)
    
    doc_vectors_final_pd = pd.DataFrame(doc_vectors_final)
    doc_vectors_final_pd.index = index_col_list
    
    return doc_vectors_final_pd

In [6]:
params4 = {"train_year_of_citations": 2020,
          "test_year_of_citations": 2021,
          "min_train_year_published":2019,
          "max_train_year_published":2020,
          "min_test_year_published":2020,
          "max_test_year_published":2020,
          "embeddings_from_year": 2019,
          "embeddings_to_year": 2020}

for params in [params4]:

    train_year_of_citations = params["train_year_of_citations"]
    test_year_of_citations = params["test_year_of_citations"]  
    min_train_year_published = params["min_train_year_published"]
    max_train_year_published = params["max_train_year_published"]
    min_test_year_published = params["min_test_year_published"]
    max_test_year_published = params["max_test_year_published"]
    embeddings_from_year = params["embeddings_from_year"]
    embeddings_to_year = params["embeddings_to_year"]
    
    #target
    train_df = add_target_opencitatins_marginal(target_year = train_year_of_citations,df = df_all,target_col_name="target")
    test_df = add_target_opencitatins_marginal(target_year = test_year_of_citations, df = df_all,target_col_name = "target")
    # Train test split of target based on year of publication
    train_df = train_df[(train_df['Year']<=max_train_year_published) & (train_df['Year']>= min_train_year_published)].set_index("doi")
    test_df = test_df[(test_df['Year']<=max_test_year_published) & (test_df['Year']>= min_test_year_published)].set_index("doi")

    # doc embeddings
    model_w2v = gensim.models.Word2Vec.load("w2v_published_only_english_between_2019 and 2020.model")
    train_df["abstract_tokenized"] = tokenized_column(train_df.reset_index()["abstract_cleaned"])
    test_df["abstract_tokenized"] = tokenized_column(test_df.reset_index()["abstract_cleaned"])
    
    # FINAL PREPARED DATA !
    df_X_train_avg = transform_to_document_vector(text_col_tokenized = train_df.reset_index().abstract_tokenized,model = model_w2v,index_col_list = list(train_df.index),agg_func = "avg").fillna(0)
    df_X_test_avg = transform_to_document_vector(text_col_tokenized = test_df.reset_index().abstract_tokenized,model = model_w2v,index_col_list = list(test_df.index),agg_func = "avg").fillna(0)
    y_train_w2v = train_df["target"]
    y_test_w2v = test_df["target"]
    
    # TRAIN MODEL
    cf_w2v_splitted_train_avg = LogisticRegression(penalty = None,random_state = 0).fit(df_X_train_avg, y_train_w2v)
    

C:\Users\berl03\AppData\Local\Temp/ipykernel_14516/433318618.py:22: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  train_df = add_target_opencitatins_marginal(target_year = train_year_of_citations,df = df_all,target_col_name="target")
b'Skipping line 171761: expected 5 fields, saw 8\nSkipping line 208878: expected 5 fields, saw 8\n'


0    22317
1    17775
Name: target, dtype: int64


C:\Users\berl03\AppData\Local\Temp/ipykernel_14516/433318618.py:23: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  test_df = add_target_opencitatins_marginal(target_year = test_year_of_citations, df = df_all,target_col_name = "target")
b'Skipping line 171761: expected 5 fields, saw 8\nSkipping line 208878: expected 5 fields, saw 8\n'


0    8576
1    4996
Name: target, dtype: int64


100%|█████████████████████████████████████████████████████████████████████████████| 8645/8645 [01:11<00:00, 120.36it/s]


In [12]:
pickle.dump(cf_w2v_splitted_train_avg, open('lreg_w2v_avg_'+str(train_year_of_citations)+'.sav', 'wb'))
df_X_train_avg.to_csv("X_train.csv")
df_X_test_avg.to_csv("X_test.csv")
y_train_w2v.reset_index().merge(train_df.reset_index()[["doi","abstract_cleaned","abstract_tokenized"]], on = "doi",how="left").to_csv("y_train.csv")
y_test_w2v.reset_index().merge(test_df.reset_index()[["doi","abstract_cleaned","abstract_tokenized"]], on = "doi",how="left").to_csv("y_test.csv")